<a href="https://colab.research.google.com/github/AndssY/test/blob/main/saving_loading_dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install stable-baselines3[extra]
!pip install pyglet==1.4

Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.13).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
import gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.ppo.policies import MlpPolicy
import numpy as np

In [43]:
env = gym.make('CartPole-v1') # 第一步注册环境

model = PPO('MlpPolicy', env, verbose=0) # 第二步导入RL模型，例如PPOor啥子的
# 第三步确定policy类，这里是MlpPolicy，也可以是其他的
# 这里可以使用stable_baselines3.ppo.policies中的policy，也可以是字符串的形式
# 字符串的话会更好一点，因为像SAC有自身的MlpPolicy，更推荐

In [44]:
print(env.action_space) # env的动作空间
print(env.observation_space) # env的观测空间
print(env.action_space.sample()) # 动作采样
# 这几个动作分别是什么意思呢？就需要去gym里去查源码了，一般都有
# 还有观测空间也可以sample，具体意思也需要去查源码

Discrete(2)
Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
0


In [45]:
h1 = gym.spaces.Box(low=-1.0, high=2.0, shape=(3, 4), dtype=np.float32)
h1.sample()
# 观测空间

array([[ 0.26951334, -0.4050963 , -0.8792691 ,  1.4273363 ],
       [ 1.3763878 ,  0.45691875, -0.68160194, -0.5690137 ],
       [ 1.5239491 ,  1.0222771 ,  1.7173711 , -0.71859616]],
      dtype=float32)

In [46]:
def evaluate(model, num_episodes=100):
  '''
  评估一个RL agent
  :param model:(BaseRLModel object) the RL Agent
  :param num_episodes:(int) number of episodes to evaluate it
  :return :(float)Mean reward for the last num_episodes
  '''
  # 这个函数只能用于single Environment
  env = model.get_env()
  all_episode_rewards = []
  for i in range(num_episodes):
    episode_rewards = []
    done = False
    obs = env.reset()
    while not done:
      action, _states = model.predict(obs)
      # env.render()
      obs, reward, done, info = env.step(action)
      episode_rewards.append(reward)
    all_episode_rewards.append(sum(episode_rewards))
  mean_episode_reward = np.mean(all_episode_rewards)
  print("Mean reward:", mean_episode_reward, "Num episodes:", num_episodes)

  return mean_episode_reward  

In [47]:
mean_reward = evaluate(model, 1000)

Mean reward: 22.194 Num episodes: 1000


In [48]:
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=1000)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:10.77 +/- 1.30


In [ ]:
# 自己编写一段简单的测试代码
episodes = 100
for ep in range(episodes):
  env.reset()
  rewards = 0
  done = False
  while not done:
    action = model.predict()
    obs, reward, done, info = env.step(action)
    rewards += reward
  print(rewards)

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
# 模型训练
model.learn(total_timesteps=int(2000))
# 模型保存
model.save('haha')
del model

In [53]:
# 模型加载
model2 = PPO.load('haha', env=env)
# 模型评估
mean_reward, std_reward = evaluate_policy(
    model2,
    model2.get_env(),
    deterministic = True,
    n_eval_episodes=10
)
print(mean_reward)

500.0


In [ ]:
# tensorboard可视化
from stable_baselines3 import DQN
model3 = DQN(
    'MlpPolicy',
    env,
    verbose = 1,
    tensorboard_log='./logs'
)
model3.learn(total_timesteps=int(10000),
        tb_log_name='test1')

In [61]:
!tensorboard --logdir ./logs --bind_all


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

TensorBoard 2.9.1 at http://9447bc5193d5:6006/ (Press CTRL+C to quit)
